In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from tqdm import tqdm
tqdm.pandas()
init_notebook_mode(connected = True)

In [2]:
df = pd.read_csv('train_mod.csv')
df['order_day'] = pd.to_datetime(df['order_time']).dt.dayofweek
df['order_hr'] = pd.to_datetime(df['order_time']).dt.hour
df['order_min'] = pd.to_datetime(df['order_time']).dt.minute
df['order_sec'] = pd.to_datetime(df['order_time']).dt.second

df.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,...,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time,order_day,order_hr,order_min,order_sec
0,2021-01-27 08:47:15,524758,2021-01-27 00:00:00,2021-01-27 08:51:01,2021-01-27 08:51:30,2021-01-27 08:59:18,2021-01-27 09:07:14,0,1.3086,0.08,...,8734.0,NaN,NaN,NaN,299.583333,NaN,2,8,47,15
1,2021-01-29 08:37:58,442759,2021-01-29 00:00:00,2021-01-29 08:37:58,2021-01-29 08:39:57,2021-01-29 08:57:45,2021-01-29 09:00:23,0,1.3477,0.03,...,8735.0,NaN,NaN,NaN,245.266667,NaN,4,8,37,58
2,2021-01-29 08:38:07,442767,2021-01-29 00:00:00,2021-01-29 09:35:08,2021-01-29 09:36:58,NaN,NaN,0,1.6314,2.22,...,8735.0,auto,Reassign,1.0,245.416667,2021-01-29 09:54:21,4,8,38,7
3,2021-02-01 14:31:33,339093,2021-02-01 00:00:00,2021-02-01 14:31:34,2021-02-01 14:33:32,2021-02-01 14:53:07,2021-02-01 15:00:36,0,0.5356,1.01,...,8737.0,NaN,NaN,NaN,646.016667,NaN,0,14,31,33
4,2021-02-01 14:54:31,341573,2021-02-01 00:00:00,2021-02-01 15:00:55,2021-02-01 15:01:01,2021-02-01 15:08:47,2021-02-01 15:16:06,0,2.5956,1.56,...,8737.0,auto,Reassignment Request from SE portal.,1.0,668.983333,NaN,0,14,54,31


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            450000 non-null  object 
 1   order_id              450000 non-null  int64  
 2   order_date            450000 non-null  object 
 3   allot_time            450000 non-null  object 
 4   accept_time           449843 non-null  object 
 5   pickup_time           447579 non-null  object 
 6   delivered_time        444782 non-null  object 
 7   rider_id              450000 non-null  int64  
 8   first_mile_distance   450000 non-null  float64
 9   last_mile_distance    450000 non-null  float64
 10  alloted_orders        433052 non-null  float64
 11  delivered_orders      432659 non-null  float64
 12  cancelled             450000 non-null  int64  
 13  undelivered_orders    432659 non-null  float64
 14  lifetime_order_count  449947 non-null  float64
 15  

In [4]:
df['lifetime_order_count'] = df['lifetime_order_count'].fillna(0)

In [5]:
call = pd.read_csv('call_data.csv', usecols=['order_id', 'reason_text', 'user_type', 'rider_id'])
call.head()

,order_id,reason_text,user_type,rider_id
0,556753,NaN,customer,11696
1,556753,NaN,customer,11696
2,556754,NaN,customer,18117
3,556755,NaN,customer,18623
4,556755,NaN,customer,18623


In [6]:
call.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562624 entries, 0 to 562623
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   order_id     562624 non-null  int64 
 1   reason_text  37061 non-null   object
 2   user_type    562624 non-null  object
 3   rider_id     562624 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 17.2+ MB


In [7]:
call['user_type'].value_counts()

customer    525563
support      37061
Name: user_type, dtype: int64

In [8]:
full_data = pd.merge(df, call, on='order_id')
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421994 entries, 0 to 421993
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            421994 non-null  object 
 1   order_id              421994 non-null  int64  
 2   order_date            421994 non-null  object 
 3   allot_time            421994 non-null  object 
 4   accept_time           421994 non-null  object 
 5   pickup_time           419858 non-null  object 
 6   delivered_time        412375 non-null  object 
 7   rider_id_x            421994 non-null  int64  
 8   first_mile_distance   421994 non-null  float64
 9   last_mile_distance    421994 non-null  float64
 10  alloted_orders        404931 non-null  float64
 11  delivered_orders      404480 non-null  float64
 12  cancelled             421994 non-null  int64  
 13  undelivered_orders    404480 non-null  float64
 14  lifetime_order_count  421994 non-null  float64
 15  

## Order cancel rate vs day of week

In [20]:
days_str = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
days = [0,1,2,3,4,5,6]
data = []
for day in days:
    cancelled = df[df['order_day'] == day]['cancelled'].values
    data+=[[days_str[day], sum(cancelled), len(cancelled) - sum(cancelled), sum(cancelled)/len(cancelled)]]
data = pd.DataFrame(data, columns = ["day", "cancelled", "delivered", 'rates'])
px.bar(data, x = 'day', y = 'rates', title = 'Cancellation Rate by Day of the Week')

## Time of the day when you have highest cancelled:orders placed ratio

In [22]:
data = df.copy()
data['order_time_24'] = data['order_hr']*100 + data['order_min'] 
data.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,...,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time,order_day,order_hr,order_min,order_sec,order_time_24
0,2021-01-27 08:47:15,524758,2021-01-27 00:00:00,2021-01-27 08:51:01,2021-01-27 08:51:30,2021-01-27 08:59:18,2021-01-27 09:07:14,0,1.3086,0.08,...,NaN,NaN,NaN,299.583333,NaN,2,8,47,15,847
1,2021-01-29 08:37:58,442759,2021-01-29 00:00:00,2021-01-29 08:37:58,2021-01-29 08:39:57,2021-01-29 08:57:45,2021-01-29 09:00:23,0,1.3477,0.03,...,NaN,NaN,NaN,245.266667,NaN,4,8,37,58,837
2,2021-01-29 08:38:07,442767,2021-01-29 00:00:00,2021-01-29 09:35:08,2021-01-29 09:36:58,NaN,NaN,0,1.6314,2.22,...,auto,Reassign,1.0,245.416667,2021-01-29 09:54:21,4,8,38,7,838
3,2021-02-01 14:31:33,339093,2021-02-01 00:00:00,2021-02-01 14:31:34,2021-02-01 14:33:32,2021-02-01 14:53:07,2021-02-01 15:00:36,0,0.5356,1.01,...,NaN,NaN,NaN,646.016667,NaN,0,14,31,33,1431
4,2021-02-01 14:54:31,341573,2021-02-01 00:00:00,2021-02-01 15:00:55,2021-02-01 15:01:01,2021-02-01 15:08:47,2021-02-01 15:16:06,0,2.5956,1.56,...,auto,Reassignment Request from SE portal.,1.0,668.983333,NaN,0,14,54,31,1454


In [24]:
bins = list(range(0, 2500, 400))
data['binned'] = np.searchsorted(bins, data['order_time_24'].values)
data = data[['cancelled', 'order_time_24', 'order_day', 'binned']]
days_str = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
days = [0,1,2,3,4,5,6]
values = [[-1 for _ in range(len(bins)-1)] for _ in range(7)]
for day in days:
    for bin in range(len(bins)-1):
        temp = data[(data['order_day'] == day) & (data['binned'] == bin)]
        try:
            rate = sum(temp['cancelled'].values)/len(temp['cancelled'].values)
            values[day][bin] = rate
        except:
            continue
temp = [str(a).zfill(4)[:2]+':'+str(a).zfill(4)[2:] for a in bins]
x = []
for i in range(len(temp)-1):
    x+=['-'.join([temp[i], temp[i+1]])]
y = days_str

text = [[str(round(i,3)) for i in values[j]] for j in range(7)]

fig = px.imshow(np.array(values)[:, 1:], x = x[1:], y = y, aspect='auto', color_continuous_scale='Viridis', text_auto=True, title = 'Cancellation Rate by Time Slot of a Day')
fig.update_traces(text=np.array(text)[:,1:], texttemplate="%{text}")
fig.show()

## Relation between the time till cancellation and the type of call made

In [27]:
data = df[df['cancelled']==1].copy()
data['accept_time'] = data['accept_time'].fillna(data['allot_time'])
data['pickup_time'] = data['pickup_time'].fillna(data['accept_time'])
data['time_to_cancel'] = pd.to_datetime(data['cancelled_time']) - pd.to_datetime(data['pickup_time'])
data['time_to_cancel'] = pd.to_timedelta(data['time_to_cancel']).astype('timedelta64[m]').astype(int)
data = data[['order_id', 'time_to_cancel']]
data.head()

,order_id,time_to_cancel
2,442767,17
84,239679,40
206,172227,1
225,401533,29
339,200588,13


In [28]:
types = []
no_id = []
for order_id in data['order_id']:
    type_ = list(call[call['order_id'] == order_id]['user_type'].values)
    if len(type_) == 0:
        no_id+=[order_id]
        continue
    types+=[max(set(type_), key=type_.count)]
data = data[~data['order_id'].isin(no_id)]
data['user_type'] = types
data

,order_id,time_to_cancel,user_type
2,442767,17,support
84,239679,40,customer
225,401533,29,customer
569,300409,25,customer
621,573391,27,customer
...,...,...,...
449982,592805,2792,support
449988,593470,121,customer
449989,593548,91,customer
449994,593623,44,support


In [29]:
px.box(data[data['time_to_cancel']<150], x = 'user_type', y='time_to_cancel', title = 'Time taken to Cancel the order based on the type of call')

## For orders cancelled after food prep, time taken to cancel from pickup time

In [15]:
data = df[df['cancelled']==1].copy()
data = data.dropna(subset = ['pickup_time'])
data = data[['pickup_time', 'cancelled_time']]
data['time_to_cancel'] = pd.to_datetime(data['cancelled_time']) - pd.to_datetime(data['pickup_time'])
data['time_to_cancel'] = pd.to_timedelta(data['time_to_cancel']).astype('timedelta64[m]').astype(int)
px.box(data[data['time_to_cancel']<200]['time_to_cancel'])

## New vs Medium vs experienced rider cancellation rate

In [16]:
riders = df['rider_id'].unique()
data = []
for rider in tqdm(riders):
    rider_data = df[df['rider_id'] == rider][['order_date', 'cancelled', 'lifetime_order_count']]
    
    lifetime_orders = max(rider_data['lifetime_order_count'])
    rate = 1-sum(rider_data['cancelled'])/len(rider_data['cancelled'])
    data+=[[lifetime_orders, rate]]
data = pd.DataFrame(data, columns = ['lifetime_order_count', 'rate'])
data

100%|████████████████████████████████████████████████████████| 19537/19537 [00:20<00:00, 976.70it/s]


,lifetime_order_count,rate
0,8740.0,0.857143
1,1392.0,1.000000
2,1421.0,1.000000
3,308.0,1.000000
4,119.0,1.000000
...,...,...
19532,0.0,0.000000
19533,282.0,1.000000
19534,55.0,0.000000
19535,31.0,1.000000


In [17]:
px.scatter(data, x = 'lifetime_order_count', y = 'rate', title='Completion Rate vs Lifetime order count')

## Cancellation Rate vs Average Session Time per day

In [18]:
riders = df['rider_id'].unique()
data = []
for rider in tqdm(riders):
    times = []
    rider_data = df[df['rider_id'] == rider][['cancelled', 'order_date', 'session_time']]
    rate = 1-sum(rider_data['cancelled'])/len(rider_data['cancelled'])
    for date in rider_data['order_date'].unique():
        rider_data_date = rider_data[rider_data['order_date'] == date]
        time = max(rider_data_date['session_time'].values)
        times+=[time]
    avg = sum(times)/len(times)
    data+=[[avg, rate]]
    
data = pd.DataFrame(data, columns=['Average Session Time', 'Completion Rate'])
data.head()
        

100%|████████████████████████████████████████████████████████| 19537/19537 [00:46<00:00, 424.72it/s]


,Average Session Time,Completion Rate
0,360.233333,0.857143
1,564.644444,1.000000
2,492.110000,1.000000
3,129.675000,1.000000
4,468.588889,1.000000


In [19]:
px.scatter(data, x = 'Average Session Time', y = 'Completion Rate', title='Completion Rate vs Average Session Time')